In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Input, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt  # Import sqrt function

# Load the data
df = pd.read_csv('mucolytics.csv')

# Convert 'Sold_date' to datetime format and set as index
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_data = df.pivot_table(values='Sold_quantity', index='Sold_date', 
                             columns='Product_details', aggfunc='sum').resample('W').sum()

# Normalize the sales data
scaler = MinMaxScaler()
weekly_data_scaled = scaler.fit_transform(weekly_data)
weekly_data_scaled = pd.DataFrame(weekly_data_scaled, index=weekly_data.index, columns=weekly_data.columns)

# Prepare data for LSTM
def create_sequences(data, sequence_length):
    x = []
    y = []
    for i in range(len(data) - sequence_length):
        x.append(data.iloc[i:i + sequence_length].values)
        y.append(data.iloc[i + sequence_length].values)
    return np.array(x), np.array(y)

sequence_length = 2  # Number of weeks used to predict the next week
x_lstm, y_lstm = create_sequences(weekly_data_scaled, sequence_length)

print("x_lstm shape:", x_lstm.shape)
print("y_lstm shape:", y_lstm.shape)

x_lstm shape: (162, 2, 49)
y_lstm shape: (162, 49)


In [11]:
lstm_model = Sequential([
    Input(shape=(sequence_length, len(weekly_data_scaled.columns))),
    LSTM(49, activation='tanh', recurrent_activation='sigmoid', return_sequences=False),
])
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the LSTM model
history = lstm_model.fit(x_lstm, y_lstm, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Extract features using LSTM
features = lstm_model.predict(x_lstm)

# Define and compile the CNN model
input_shape = (features.shape[1], 1)
cnn_model_enhanced = Sequential([
    Input(shape=input_shape),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(len(weekly_data.columns))
])
cnn_model_enhanced.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Fit the CNN model
history_cnn = cnn_model_enhanced.fit(features, y_lstm, epochs=100, batch_size=32, validation_split=0.2)

# Predictions made by the CNN model
scaled_predictions = cnn_model_enhanced.predict(features)

# Inverse transform the scaled predictions to original scale
actual_predictions = scaler.inverse_transform(scaled_predictions)

# Create a DataFrame for the forecasted data
forecast_df = pd.DataFrame(actual_predictions, index=weekly_data.index[sequence_length:], columns=weekly_data.columns)

# Reindex the forecast DataFrame to ensure it includes the required date range
forecast_df = forecast_df.reindex(pd.date_range(start=weekly_data.index[sequence_length], end=weekly_data.index[-1], freq='W'))

# Display the forecast for the week of February 5 to 11, 2024
forecast_week = forecast_df.loc['2024-02-04']  # The week starting on February 4, 2024
sorted_forecast_week = forecast_week.sort_values(ascending=False)  # Sort values from highest to lowest

# Convert to DataFrame and save the sorted forecast to Excel
sorted_forecast_week_df = sorted_forecast_week.to_frame(name='Sold_quantity')
sorted_forecast_week_df.index.name = 'Product_details'
sorted_forecast_week_df.reset_index(inplace=True)
sorted_forecast_week_df.to_excel('mucolytics1_sorted_by_quantity.xlsx', index=False)

# Display RMSE
print("Root Mean Squared Error (RMSE):", history_cnn.history['root_mean_squared_error'])

# Calculate and print RMSE in original units
rmse_original = sqrt(mean_squared_error(y_lstm, actual_predictions))
print("RMSE on original scale:", rmse_original)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0392 - val_loss: 0.0609
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0373 - val_loss: 0.0581
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0342 - val_loss: 0.0557
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0338 - val_loss: 0.0536
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0349 - val_loss: 0.0519
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0318 - val_loss: 0.0504
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0316 - val_loss: 0.0492
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0299 - val_loss: 0.0481
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0284 - val_loss: 0.0472
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0290 - val_loss: 0.0465
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0295 - val_loss: 0.0459
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0282 - val_loss: 0.0455


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0204 - val_loss: 0.0449
Epoch 71/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0201 - val_loss: 0.0451
Epoch 72/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0201 - val_loss: 0.0452
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0209 - val_loss: 0.0452
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0187 - val_loss: 0.0453
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0187 - val_loss: 0.0454
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0190 - val_loss: 0.0454
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0190 - val_loss: 0.0453
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0195 - val_loss: 0.0453
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0192 - val_loss: 0.0452
Epoch 80/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0195 - val_loss: 0.0451
Epoch 81/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0199 - val_loss: 0.0450
Epoch

Epoch 21/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0247 - root_mean_squared_error: 0.1572 - val_loss: 0.0446 - val_root_mean_squared_error: 0.2113
Epoch 22/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0249 - root_mean_squared_error: 0.1579 - val_loss: 0.0446 - val_root_mean_squared_error: 0.2112
Epoch 23/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0255 - root_mean_squared_error: 0.1596 - val_loss: 0.0443 - val_root_mean_squared_error: 0.2105
Epoch 24/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0248 - root_mean_squared_error: 0.1575 - val_loss: 0.0443 - val_root_mean_squared_error: 0.2106
Epoch 25/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0233 - root_mean_squared_error: 0.1528 - val_loss: 0.0441 - val_root_mean_squared_error: 0.2100
Epoch 26/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0228 - root_mean_squared_error: 0.1509 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2087
Epoch 27/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0230

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0201 - root_mean_squared_error: 0.1418 - val_loss: 0.0458 - val_root_mean_squared_error: 0.2140
Epoch 65/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0204 - root_mean_squared_error: 0.1427 - val_loss: 0.0460 - val_root_mean_squared_error: 0.2146
Epoch 66/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0206 - root_mean_squared_error: 0.1435 - val_loss: 0.0454 - val_root_mean_squared_error: 0.2130
Epoch 67/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0208 - root_mean_squared_error: 0.1441 - val_loss: 0.0453 - val_root_mean_squared_error: 0.2129
Epoch 68/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0199 - root_mean_squared_error: 0.1409 - val_loss: 0.0456 - val_root_mean_squared_error: 0.2136
Epoch 69/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0193 - root_mean_squared_error: 0.1391 - val_loss: 0.0461 - val_root_mean_squared_error: 0.2147
Epoch 70/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0207 - root_mean_